In [1]:
from maap.maap import MAAP
maap = MAAP(maap_host='api.ops.maap-project.org')

# Launch DPS for extract_filter_atl08.py
using the rebinned 30m h5 granules

In [2]:
from os import path
import os, glob

In [3]:
curr_dir = wk_dir = os.path.dirname(os.path.realpath('__file__'))
print(curr_dir)

/projects/icesat2_boreal/dps/alg_2-3


## Test extract_filter_atl08.py code on Test Data
We are running extract_filter_atl08.py (but *FILTERING* is turned off):

python /projects/icesat2_boreal/lib/extract_filter_atl08.py -o "/projects/test_data/test_data_30m" -i "/projects/my-private-bucket/dps_output/run_rebinning_ubuntu/master/2022/02/25/18/51/17/542274/ATL08_30m_20190721220156_03640403_005_01.h5" --no-filter-qual --do_30m

python /projects/icesat2_boreal/lib/extract_filter_atl08.py -o "/projects/test_data/test_data_30m" -i "https://maap-ops-workspace.s3.amazonaws.com.com/lduncanson/dps_output/run_rebinning_ubuntu/master/2022/02/25/18/51/17/542274/ATL08_30m_20190721220156_03640403_005_01.h5" --no-filter-qual --do_30m

python lib/extract_filter_atl08.py -i "path/to/h5file" -o "path/of/out/dir" --no-filter-qual --do_30m

### Build the input list of granules
##### replace the local path with the https url

In [10]:
%%time
INPUT_ATL08_GRANULE_LIST = glob.glob("/projects/my-private-bucket/dps_output/run_rebinning_ubuntu/master/2022/**/*.h5", recursive=True)
INPUT_ATL08_GRANULE_LIST = [i.replace("/projects/my-private-bucket", "https://maap-ops-workspace.s3.amazonaws.com/lduncanson") for i in INPUT_ATL08_GRANULE_LIST]
len(INPUT_ATL08_GRANULE_LIST)

CPU times: user 317 ms, sys: 403 ms, total: 720 ms
Wall time: 4min 43s


1107

#### Set TEST, get list

In [46]:
TEST_DPS  = True

if TEST_DPS:
    DPS_INPUT_ATL08_GRANULE_LIST = INPUT_ATL08_GRANULE_LIST[-10:]
else:
    DPS_INPUT_ATL08_GRANULE_LIST = INPUT_ATL08_GRANULE_LIST
    
DPS_INPUT_ATL08_GRANULE_LIST[-1]

'https://maap-ops-workspace.s3.amazonaws.com/lduncanson/dps_output/run_rebinning_ubuntu/master/2022/02/26/00/11/54/852581/ATL08_30m_20181014003519_02350106_005_01.h5'

## Run a DPS job across the list

In [44]:
for i, INPUT_ATL08_GRANULE in enumerate(DPS_INPUT_ATL08_GRANULE_LIST):

    DPS_num = i+1
    in_param_dict = {
                        'input_file': INPUT_ATL08_GRANULE
                    }

    submit_result = maap.submitJob(
            identifier='run_extract_filter_atl08',
            algo_id='run_extract_filter_atl08_ubuntu',
            version='master',
            username='lduncanson', # username needs to be the same as whoever created the workspace
            queue='maap-dps-worker-8gb',
            **in_param_dict
        )

    #submit_result = 'submit test'
    if DPS_num in [1, 100, 500, 1000, 3000, len(INPUT_ATL08_GRANULE_LIST)]:
       print(f"DPS run #: {DPS_num} | granule name: {os.path.basename(INPUT_ATL08_GRANULE)} | job info: {submit_result}") 

DPS run #: 1 | granule name: ATL08_30m_20210713125446_03041202_005_01.h5 | job info: {'status': 'success', 'http_status_code': 200, 'job_id': 'ab4503c2-c65b-41e5-80f2-ecea558f6524'}
DPS run #: 10 | granule name: ATL08_30m_20181014003519_02350106_005_01.h5 | job info: {'status': 'success', 'http_status_code': 200, 'job_id': '2a02cb77-21b1-4f5b-a80e-71f53ee19900'}


In [48]:
os.system( 'python /projects/icesat2_boreal/lib/build_tindex_master.py -t ATL08 -dps_year 2021 --outdir /projects/my-public-bucket/test_output')

512